## 图像质量评估指标公式与参数解释


### 关键特性对比
| 指标   | 评估对象       | 输入类型       | 值域方向      | 典型应用场景       |
|--------|----------------|----------------|---------------|--------------------|
| SSIM   | 单对图像       | 像素级对比     | 值越高越好    | 图像修复/超分辨率 |
| FID    | 数据分布       | 特征空间分布   | 值越低越好    | GAN模型评估       |

---

### 1. SSIM (结构相似性指数)

#### 公式
$$
\text{SSIM}(x, y) = \frac{(2\mu_x \mu_y + C_1)(2\sigma_{xy} + C_2)}{(\mu_x^2 + \mu_y^2 + C_1)(\sigma_x^2 + \sigma_y^2 + C_2)}
$$

#### 参数说明
- $\mu_x, \mu_y$: 图像 $x$ 和 $y$ 的**局部均值**  
- $\sigma_x^2, \sigma_y^2$: 图像 $x$ 和 $y$ 的**局部方差**  
- $\sigma_{xy}$: 图像 $x$ 和 $y$ 之间的**协方差**  
- $C_1, C_2$: 防止分母为零的**稳定常数**，通常定义为：
  - $C_1 = (k_1 L)^2$,  $C_2 = (k_2 L)^2$  
  - 其中 $L$ 是像素值的动态范围（如 `255` 对应 8-bit 图像）  
  - $k_1 = 0.01$, $k_2 = 0.03$（默认值）

---

### 2. FID (Frechet Inception Distance)

#### 公式
$$
\text{FID} = \|\mu_P - \mu_Q\|^2 + \text{Tr}\left( \Sigma_P + \Sigma_Q - 2 (\Sigma_P \Sigma_Q)^{1/2} \right)
$$

#### 参数说明
- $\mu_P, \mu_Q$: 真实数据分布 $P$ 和生成数据分布 $Q$ 的**特征均值向量**  
  - 通过预训练模型（如 Inception-v3）提取的特征向量的均值
- $\Sigma_P, \Sigma_Q$: 真实数据分布 $P$ 和生成数据分布 $Q$ 的**协方差矩阵**  
  - 描述特征向量之间的相关性
- $\text{Tr}(\cdot)$: 矩阵的**迹**（对角线元素之和）  
- $(\Sigma_P \Sigma_Q)^{1/2}$: 协方差矩阵乘积的**平方根矩阵**  
  - 需通过数值方法（如 SVD）计算实数解

In [ ]:
from torchvision import transforms
from model.RFID_Dataset import build_class_datasets
from model.Normalization import RobustNorm

train_transform=transforms.Compose([
    RobustNorm(-68.0, 68.0),
    transforms.Resize(size=(299,299)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])
# train_transform=None

# 训练集
train_datasets=build_class_datasets(
    r"data\RFID_multi_628\dataset\train",
    T=32,
    step=1,
    num_channels=3,
    transforms=train_transform,
)
for label,dataset in train_datasets.items():
    print(f"label {label} 数据个数为: {len(dataset)}")

In [ ]:
from torchvision import transforms
from model.RFID_Dataset import build_class_datasets
from model.Normalization import RobustNorm

eval_transform=transforms.Compose([
    RobustNorm(-1.0, 1.0),
    transforms.Resize(size=(299,299)),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
])
# eval_transform=None

# 测试集
test_datasets=build_class_datasets(
    # r"data\Temp\dataset\all",
    r"./output/base",
    T=32,
    step=32,
    num_channels=3,
    transforms=eval_transform,
)
for label,dataset in test_datasets.items():
    print(f"label {label} 数据个数为: {len(dataset)}")

In [ ]:
## 使用自实现方法计算FID

from utils.SimilarityUtils import *
from utils.ConfigUtils import get_classes

classes=list(get_classes(r"data\RFID_multi_628\data.yml").values())
num_classes = len(classes)

num_classes=6
for i in range(num_classes):
    print(f"label {i}: {classes[i]}")
    
    fid_train_dataset=train_datasets.get(i)
    fid_test_dataset=test_datasets.get(i)
    if not all ([fid_train_dataset, fid_test_dataset]):
        print("数据集不存在")
        continue

    # 计算FID分数
    fid_score=execute_fid_metric_temp(
        fid_train_dataset, 
        fid_test_dataset,
        # model=model
    )
    print(f"FID: {fid_score:.6f}")
    print ("="*30)
    

In [ ]:
from torchvision import transforms
from model.RFID_Dataset import RFID_Dataset
from model.Normalization import RobustNorm

train_transform = transforms.Compose(
    [
        RobustNorm(-68.0, 68.0),
        transforms.Resize(size=(299, 299)),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    ]
)

# 训练集
train_datasets = RFID_Dataset(
    r"data\RFID_multi_628\dataset\train",
    T=32,
    step=1,
    num_channels=3,
    transform=train_transform,
)

eval_transform = transforms.Compose(
    [
        RobustNorm(-1.0, 1.0),
        transforms.Resize(size=(299, 299)),
        transforms.Lambda(lambda x: x.repeat(3, 1, 1)),
    ]
)
# eval_transform=None

# 测试集
test_datasets = RFID_Dataset(
    # r"data\Temp\dataset\all",
    r"./output/base",
    T=32,
    step=32,
    num_channels=3,
    transform=eval_transform,
)

fid_score = execute_fid_metric_temp(
    train_datasets,
    test_datasets,
)
print(f"FID: {fid_score:.6f}")
print("=" * 30)

In [ ]:
## 使用torchmetrics计算FID

from utils.SimilarityUtils import *
from torchmetrics.image.fid import FrechetInceptionDistance

fid_instance=FrechetInceptionDistance(
    normalize=True
)

num_classes=3
for i in range(num_classes):
    print(f"label: {i}")
    
    fid_train_dataset=train_datasets.get(i)
    fid_test_dataset=test_datasets.get(i)
    if not all ([fid_train_dataset, fid_test_dataset]):
        print("数据集不存在")
        continue

    # 计算FID分数
    fid_score=execute_fid_metric(
        fid_train_dataset, 
        fid_test_dataset,
        fid_instance=fid_instance
    )
    print(f"FID: {fid_score:.6f}")
    print ("="*30)